In [1]:
import numpy as np
import random
import torch
import re
import os
import jieba
import time
from torch import nn
import torch.nn.functional as F
from torch.amp import autocast, GradScaler
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from collections import defaultdict
from datasets import load_dataset

In [2]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

device = torch.device('mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu')
print(f"device: {device}")

device: cuda


In [3]:
data_size = 200000

zh_path = f"data/zh_train_{data_size}.txt"
en_path = f"data/en_train_{data_size}.txt"

zh_data = []
en_data = []

if not os.path.exists(zh_path) or not os.path.exists(en_path):
    zh_en_data = load_dataset("wmt19", "zh-en", split=f"train[:{data_size}]")
    print(zh_en_data.shape)
    print(zh_en_data.cache_files)
    with open(zh_path, "w") as f:
        for i in zh_en_data:
            zh_text = i['translation']['zh']
            f.write(zh_text + "\n")
            zh_data.append(zh_text)
    with open(en_path, "w") as f:
        for i in zh_en_data:
            en_text = i['translation']['en']
            f.write(en_text + "\n")
            en_data.append(en_text)
else:
    with open(zh_path, "r") as f:
        zh_data = f.readlines()
    with open(en_path, "r") as f:
        en_data = f.readlines()

In [4]:
filter_max_len = 50
unk_freq = 2


def tokenize_zh(text):
    text = text.replace("，", ",").replace("。", ".").replace("！", "!").replace("？", "?")
    text = text.replace("[", "(").replace("]", ")").replace("（", "(").replace("）", ")").replace(" ", "")
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9,.!?()]+', '', text)
    text = text.lower()
    return jieba.lcut(text)


def tokenize_en(text):
    text = text.lower()
    return re.findall(r"\w+|[\.\,\?\!\(\)]", text)


def build_vocabs(data_tokenized):
    vocabs = ['<pad>', '<sos>', '<eos>', '<unk>']
    vocab_freq = defaultdict(int)
    for seq in data_tokenized:
        for v in seq:
            vocab_freq[v] += 1

    for v, freq in vocab_freq.items():
        if freq >= unk_freq:
            vocabs.append(v)

    stoi = {s: i for i, s in enumerate(vocabs)}
    itos = {i: s for s, i in stoi.items()}
    vocab_size = len(stoi)

    return stoi, itos, vocab_size, vocab_freq


def encode(seq, stoi, add_sos=True, add_eos=True):
    indices = [stoi[c] if c in stoi else stoi['<unk>'] for c in seq]
    if add_sos:
        indices = [stoi['<sos>']] + indices
    if add_eos:
        indices = indices + [stoi['<eos>']]
    return indices


def decode(indices, itos):
    def decode_token(t):
        if isinstance(t, int) and t != -1:
            return itos[t]
        elif isinstance(t, torch.Tensor) and t.item() != -1:
            return itos[t.item()]
        else:
            return 'X'
    result = [decode_token(idx) for idx in indices]
    return "|".join(result)


def build_dataset(tokenized):
    size = len(tokenized)
    xs = torch.zeros((size, max_en_len), dtype=torch.long)
    ys = torch.zeros((size, max_zh_len), dtype=torch.long)

    for i, content in enumerate(tokenized):
        en_encoded = encode(content[0], en_stoi)
        zh_encoded = encode(content[1], zh_stoi)
        xs[i, :len(en_encoded)] = torch.tensor(en_encoded)
        ys[i, :len(zh_encoded)] = torch.tensor(zh_encoded)

    return xs, ys


zh_tokenized = []
en_tokenized = []
max_zh_len = 0
max_en_len = 0


for text_zh, text_en in zip(zh_data, en_data):
    zh = tokenize_zh(text_zh)
    en = tokenize_en(text_en)
    if len(zh) > filter_max_len or len(en) > filter_max_len:
        continue

    zh_tokenized.append(zh)
    en_tokenized.append(en)
    if len(zh) > max_zh_len:
        max_zh_len = len(zh)
    if len(en) > max_en_len:
        max_en_len = len(en)

print("data length:", len(zh_tokenized))

max_zh_len += 2  # sos eos
max_en_len += 2  # sos eos
print("max zh len:", max_zh_len)
print("max en len:", max_en_len)
max_len = max(max_zh_len, max_en_len)

en_stoi, en_itos, en_vocab_size, en_freq = build_vocabs(en_tokenized)
zh_stoi, zh_itos, zh_vocab_size, zh_freq = build_vocabs(zh_tokenized)
print("en_vocab_size:", en_vocab_size)
print("zh_vocab_size:", zh_vocab_size)


tokenized = list(zip(en_tokenized, zh_tokenized))
xs, ys = build_dataset(tokenized)
dataset = TensorDataset(xs, ys)
g = torch.Generator().manual_seed(42)
train_size = int(len(dataset) * 0.9)
test_size = len(dataset) - train_size
train_set, test_set = random_split(dataset, [train_size, test_size], generator=g)
print("train len:", len(train_set))
print("test len:", len(test_set))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.542 seconds.
Prefix dict has been built successfully.


data length: 193330
max zh len: 52
max en len: 52
en_vocab_size: 32310
zh_vocab_size: 44129
train len: 173997
test len: 19333


In [5]:
class Head(nn.Module):
    def __init__(self, n_embd, head_size, masking=True):
        super().__init__()
        self.query = nn.Linear(n_embd, head_size)
        self.key = nn.Linear(n_embd, head_size)
        self.value = nn.Linear(n_embd, head_size)
        self.masking = masking
        if masking:
            self.register_buffer("mask", torch.tril(torch.ones(2048, 2048)))

    def forward(self, x, encoder_output=None, src_padding_mask=None):
        # x: (B, T, n_embd)
        # encoder_output: (B, T1, n_embd)
        # src_padding_mask: (B, T)

        B, T = x.shape[0], x.shape[1]

        q = self.query(x)  # (B, T, head_size)
        if encoder_output is not None:
            # CROSS-ATTENTION
            # encoder_output: (B, T1, n_embd)
            k = self.key(encoder_output)  # (B, T1, head_size)
            v = self.value(encoder_output)  # (B, T1, head_size)
        else:
            # SELF-ATTENTION
            k = self.key(x)
            v = self.value(x)
        attn = q @ k.transpose(-2, -1)  # (B, T, T) or (B, T, T1)
        attn = attn * (k.size(-1) ** -0.5)
        if self.masking:
            attn = attn.masked_fill(self.mask[:T, :T] == 0, float('-inf'))

        if src_padding_mask is not None:
            if encoder_output is not None:
                attn = attn.masked_fill(src_padding_mask.unsqueeze(1), float('-inf'))
            else:
                attn = attn.masked_fill(src_padding_mask.unsqueeze(1).expand(-1, T, -1), float('-inf'))

        attn_weights = F.softmax(attn, dim=-1)  # (B, T, T)

        out = attn_weights @ v  # (B, T, head_size)
        return out, attn_weights


class MultiHead(nn.Module):
    def __init__(self, n_embd, head_size, n_head, masking=True):
        super().__init__()
        self.heads = nn.ModuleList([Head(n_embd, head_size, masking) for _ in range(n_head)])
        self.fc = nn.Linear(n_head * head_size, n_embd)
        self.dropout = nn.Dropout(0.3)

    def forward(self, x, encoder_output=None, src_padding_mask=None, return_attn=False):
        attn_outs = [head(x, encoder_output, src_padding_mask) for head in self.heads]
        attns = [out[0] for out in attn_outs]
        out = torch.cat(attns, dim=-1)  # (B, T, n_head * head_size)
        out = self.dropout(self.fc(out))  # (B, T, n_embd)
        if return_attn:
            attn_weights = [out[1] for out in attn_outs]
            return out, attn_weights
        else:
            return out


class FeedForward(nn.Module):
    def __init__(self, n_embd, dropout=0.3):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.GELU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)


class EncoderBlock(nn.Module):
    def __init__(self, n_embd, head_size, n_head, dropout=0.3):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.sa = MultiHead(n_embd, head_size, n_head, masking=False)
        self.ln2 = nn.LayerNorm(n_embd)
        self.ff = FeedForward(n_embd, dropout)

    def forward(self, x, src_padding_mask=None, return_attn=False):
        if return_attn:
            sa_out, attn_weights = self.sa(self.ln1(x), src_padding_mask=src_padding_mask, return_attn=True)
        else:
            sa_out = self.sa(self.ln1(x), src_padding_mask=src_padding_mask)
        x = x + sa_out
        x = x + self.ff(self.ln2(x))
        return (x, attn_weights) if return_attn else x


class DecoderBlock(nn.Module):
    def __init__(self, n_embd, head_size, n_head, dropout=0.3):
        super().__init__()
        self.ln1 = nn.LayerNorm(n_embd)
        self.sa = MultiHead(n_embd, head_size, n_head, masking=True)
        self.ln2 = nn.LayerNorm(n_embd)
        self.ca = MultiHead(n_embd, head_size, n_head, masking=False)
        self.ln3 = nn.LayerNorm(n_embd)
        self.ff = FeedForward(n_embd, dropout)

    def forward(self, x, encoder_output, src_padding_mask=None, return_attn=False):
        # x: (B, T, n_embd)
        # encoder_output: (B, T1, n_embd)
        x = x + self.sa(self.ln1(x))
        if return_attn:
            ca_out, attn_weights = self.ca(self.ln2(x), encoder_output, src_padding_mask, return_attn=True)
        else:
            ca_out = self.ca(self.ln2(x), encoder_output, src_padding_mask)
            attn_weights = None
        x = x + ca_out
        x = x + self.ff(self.ln3(x))
        return (x, attn_weights) if return_attn else x


class Model(nn.Module):
    def __init__(self, n_embd, n_head, n_layers, src_vocab_size, tgt_vocab_size, max_len, dropout=0.3):
        super().__init__()
        head_size = n_embd // n_head
        self.src_embedding = nn.Embedding(src_vocab_size, n_embd)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, n_embd)
        self.pos_embedding = nn.Embedding(max_len, n_embd)
        self.dropout = nn.Dropout(dropout)
        self.encoder_blocks = nn.ModuleList([EncoderBlock(n_embd, head_size, n_head, dropout) for _ in range(n_layers)])
        self.decoder_blocks = nn.ModuleList([DecoderBlock(n_embd, head_size, n_head, dropout) for _ in range(n_layers)])
        self.ln_in = nn.LayerNorm(n_embd)
        self.ln_tgt_in = nn.LayerNorm(n_embd)
        self.ln_f = nn.LayerNorm(n_embd)

    def encode(self, src, return_attn=False):
        B, T1 = src.shape
        src_padding_mask = (src == 0).to(src.device)  # (B, T1)

        # ENCODER
        tok_emb = self.src_embedding(src)  # (B, T1, n_embd)
        pos = torch.arange(0, T1, device=src.device).unsqueeze(0).repeat(B, 1)  # (B, T1)
        pos_emb = self.pos_embedding(pos)  # (B, T1, n_embd)
        x = self.ln_in(tok_emb + pos_emb)  # (B, T1, n_embd)
        x = self.dropout(x)

        self_attns = []
        # Pass padding mask to each encoder block
        for encoder_block in self.encoder_blocks:
            if return_attn:
                x, attn_weights = encoder_block(x, src_padding_mask, return_attn=True)
                self_attns.append(attn_weights)
            else:
                x = encoder_block(x, src_padding_mask)
        encoder_output = self.ln_f(x)  # (B, T1, n_embd)
        return (encoder_output, src_padding_mask, self_attns) if return_attn else (encoder_output, src_padding_mask)

    def decode(self, tgt, encoder_output, src_padding_mask, return_attn=False):
        B, T2 = tgt.shape
        tgt_tok_emb = self.tgt_embedding(tgt)  # (B, T2, n_embd)
        pos = torch.arange(0, T2, device=tgt.device).unsqueeze(0).repeat(B, 1)  # (B, T2)
        pos_emb = self.pos_embedding(pos)  # (B, T2, n_embd)
        x = self.ln_tgt_in(tgt_tok_emb + pos_emb)  # (B, T2, n_embd)
        x = self.dropout(x)

        cross_attns = []
        for decoder_block in self.decoder_blocks:
            if return_attn:
                x, attn_weights = decoder_block(x, encoder_output, src_padding_mask, return_attn=True)
                cross_attns.append(attn_weights)
            else:
                x = decoder_block(x, encoder_output, src_padding_mask)
        x = self.ln_f(x)  # (B, T2, n_embd)
        logits = F.linear(x, self.tgt_embedding.weight)  # (B, T2, tgt_vocab_size)
        return (logits, cross_attns) if return_attn else logits

    def forward(self, src, tgt):
        # src: (B, T1), tgt: (B, T2)
        B = src.shape[0]
        encoder_output, src_padding_mask = self.encode(src)
        decoder_target = torch.cat([tgt[:, 1:], torch.zeros(B, 1, dtype=tgt.dtype, device=tgt.device)], dim=1)
        logits = self.decode(tgt, encoder_output, src_padding_mask)  # (B, T2, tgt_vocab_size)
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), decoder_target.view(-1), ignore_index=0)
        return logits, loss

In [6]:
class EarlyStopping:
    def __init__(self, patience=5, verbose=False, min_delta=0):
        self.patience = patience
        self.verbose = verbose
        self.min_delta = min_delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0
        self.improvement_counter = 0

    def __call__(self, score):
        if self.best_score is None:
            self.best_score = score
        elif score >= self.best_score - self.min_delta:
            self.improvement_counter = 0
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.improvement_counter += 1
            if self.improvement_counter >= 3:
                self.counter = 0

In [7]:
n_embd = 128
n_head = 4
n_layers = 6
dropout = 0.3
batch_size = 128
lr = 1e-3
weight_decay = 0.01

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=4)
print(f"train_length: {len(train_set)}, test_length: {len(test_set)}, batch_size: {batch_size}, filter_max_len: {filter_max_len}, unk_freq: {unk_freq}")
print(f"lr: {lr}, weight_decay: {weight_decay}")
print(f"n_embd: {n_embd}, n_head: {n_head}, n_layers: {n_layers}, src_vocab_size: {en_vocab_size}, tgt_vocab_size: {zh_vocab_size}, max_len: {max(max_en_len, max_zh_len)}, dropout: {dropout}")

model = Model(n_embd, n_head, n_layers, en_vocab_size, zh_vocab_size, max(max_en_len, max_zh_len), dropout)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=1,
    min_lr=1e-6
)
scaler = GradScaler()

train_length: 173997, test_length: 19333, batch_size: 128, filter_max_len: 50, unk_freq: 2
lr: 0.001, weight_decay: 0.01
n_embd: 128, n_head: 4, n_layers: 6, src_vocab_size: 32310, tgt_vocab_size: 44129, max_len: 52, dropout: 0.3


In [8]:
def save_checkpoint(model, optimizer, scheduler, epoch, avg_train_loss, test_loss, path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'train_loss': avg_train_loss,
        'test_loss': test_loss,
        'en_stoi': en_stoi,
        'en_itos': en_itos,
        'zh_stoi': zh_stoi,
        'zh_itos': zh_itos,
        'model_config': {
            'n_embd': n_embd,
            'n_head': n_head,
            'n_layers': n_layers,
            'src_vocab_size': en_vocab_size,
            'tgt_vocab_size': zh_vocab_size,
            'max_len': max(max_en_len, max_zh_len),
            'dropout': dropout
        }
    }
    torch.save(checkpoint, path)


def load_checkpoint(path, device):
    checkpoint = torch.load(path, map_location=device)

    # 重建模型
    model = Model(**checkpoint['model_config'])
    model = model.to(device)

    # 将模型状态移动到正确的设备
    if device.type == 'cuda':
        checkpoint['model_state_dict'] = {k: v.to(device) for k, v in checkpoint['model_state_dict'].items()}

    model.load_state_dict(checkpoint['model_state_dict'])
    en_stoi = checkpoint['en_stoi']
    en_itos = checkpoint['en_itos']
    zh_stoi = checkpoint['zh_stoi']
    zh_itos = checkpoint['zh_itos']
    max_len = checkpoint['model_config']['max_len']

    return model, checkpoint, en_stoi, en_itos, zh_stoi, zh_itos, max_len


def print_progress(epoch, progress, loss, epoch_time):
    bar_length = 30  # Length of the progress bar
    filled_length = int(bar_length * progress // 100)
    bar = '━' * filled_length + '─' * (bar_length - filled_length)
    time_str = f"{epoch_time:.1f}s" if epoch_time < 60 else f"{epoch_time/60:.1f}min"
    print(f"epoch {epoch}, progress |{bar}| {progress:.0f}%, loss {loss.item():.4f}, time: {time_str}", end='\r')


def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0
    total_samples = 0
    with torch.no_grad():
        for x, y in data_loader:
            x = x.to(device)
            y = y.to(device)
            with autocast(device.type):
                logits, loss = model(x, y)
            total_loss += loss.item() * x.size(0)
            total_samples += x.size(0)
    model.train()
    return total_loss / total_samples

In [ ]:
epoch = 0
start_time = time.time()

In [9]:
max_epoch = 300
early_stopping = EarlyStopping(patience=5, min_delta=0.001, verbose=True)

while epoch < max_epoch:
    epoch_start_time = time.time()  # 记录每个epoch开始时间
    model.train()
    train_losses = []
    for i, batch in enumerate(train_loader):
        x = batch[0].to(device)
        y = batch[1].to(device)
        with autocast(device.type):
            out, loss = model(x, y)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        train_losses.append(loss.item())
        if i % (len(train_loader) // 20) == 0 and i > 0:
            epoch_time = time.time() - epoch_start_time
            print_progress(epoch, (i + 1) / len(train_loader) * 100, loss, epoch_time)

    test_loss = evaluate(model, test_loader, device)
    avg_train_loss = sum(train_losses) / len(train_losses)  # Calculate average training loss
    epoch_time = time.time() - epoch_start_time  # 计算每个epoch耗时
    total_time = time.time() - start_time  # 计算总耗时
    print(" " * 90, end="\r")

    # 更新学习率调度器
    scheduler.step(test_loss)

    # 打印训练信息，包含时间
    epoch_time_str = f"{epoch_time:.1f}s" if epoch_time < 60 else f"{epoch_time/60:.1f}min"
    total_time_str = f"{total_time:.1f}s" if total_time < 60 else f"{total_time/60:.1f}min"
    print(f"epoch {epoch}, avg train loss: {avg_train_loss:.4f}, test loss: {test_loss:.4f}, "
          f"lr: {scheduler.get_last_lr()[0]:.5f}, epoch time: {epoch_time_str}, total time: {total_time_str}")

    if (epoch + 1) % 10 == 0:
        save_path = f"checkpoint/transformer_checkpoint_epoch_{epoch+1}.pt"
        save_checkpoint(model, optimizer, scheduler, epoch, avg_train_loss, test_loss, save_path)
        print(f"检查点已保存到: {save_path}")

    early_stopping(test_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered. Training has been stopped.")
        break
    epoch += 1

epoch 0, avg train loss: 8.9091, test loss: 6.7136, lr: 0.00100, epoch time: 2.8min, total time: 2.8min
epoch 1, avg train loss: 6.6046, test loss: 6.2527, lr: 0.00100, epoch time: 2.8min, total time: 5.6min
epoch 2, avg train loss: 6.1442, test loss: 5.7623, lr: 0.00100, epoch time: 2.9min, total time: 8.6min
epoch 3, avg train loss: 5.7105, test loss: 5.3320, lr: 0.00100, epoch time: 2.8min, total time: 11.3min
epoch 4, avg train loss: 5.3305, test loss: 4.9131, lr: 0.00100, epoch time: 2.9min, total time: 14.3min
epoch 5, avg train loss: 4.9941, test loss: 4.5986, lr: 0.00100, epoch time: 2.8min, total time: 17.1min
epoch 6, avg train loss: 4.7104, test loss: 4.3386, lr: 0.00100, epoch time: 2.9min, total time: 20.0min
epoch 7, avg train loss: 4.4719, test loss: 4.1262, lr: 0.00100, epoch time: 2.8min, total time: 22.8min
epoch 8, avg train loss: 4.2692, test loss: 3.9199, lr: 0.00100, epoch time: 2.9min, total time: 25.7min
epoch 9, avg train loss: 4.0939, test loss: 3.7685, lr: 0.

In [10]:
save_path = "checkpoint/transformer_checkpoint.pt"
save_checkpoint(model, optimizer, scheduler, epoch, avg_train_loss, test_loss, save_path)
print(f"\n模型已保存到: {save_path}")


模型已保存到: checkpoint/transformer_checkpoint.pt


In [11]:
def top_k(model, src, sos_token, eos_token, max_len=50, device='cpu', top_k=5, temperature=0.5):
    model.eval()
    with torch.no_grad():
        src = src.to(device)
        B, T = src.shape[0], src.shape[1]
        encoder_output, src_padding_mask = model.encode(src)
        sequences = torch.ones(B, 1, dtype=torch.long, device=device) * sos_token

        for _ in range(max_len):
            logits = model.decode(sequences, encoder_output, src_padding_mask)
            logits = logits[:, -1, :] / temperature  # (B, vocab_size)
            topk_logits, top_k_indices = torch.topk(logits, k=min(top_k, logits.size(-1)))  # (B, k)
            probs = F.softmax(topk_logits, dim=-1)  # (B, k)
            next_token_idx = torch.multinomial(probs, num_samples=1)
            next_token = torch.gather(top_k_indices, 1, next_token_idx)
            sequences = torch.cat([sequences, next_token], dim=1)
            if (next_token == eos_token).all():
                break
    model.train()
    return sequences


def beam_search(model, src, sos_token, eos_token, beam_width=5, max_len=50, device='cpu'):
    model.eval()
    with torch.no_grad():
        src = src.to(device)
        B, T = src.shape[0], src.shape[1]
        encoder_output, src_padding_mask = model.encode(src)
        outputs = []
        for batch_idx in range(B):
            enc_output = encoder_output[batch_idx:batch_idx + 1]  # (1, T1, n_embd)
            src_mask = src_padding_mask[batch_idx:batch_idx + 1]  # (1, T1)
            beams = [(torch.ones(1, 1, dtype=torch.long, device=device) * sos_token, 0.0)]
            for _ in range(max_len):
                new_beams = []
                for seq, log_prob in beams:
                    if seq[0, -1] == eos_token:
                        new_beams.append((seq, log_prob))
                        continue
                    logits = model.decode(seq, enc_output, src_mask)
                    logits = logits[:, -1, :]  # (1, vocab_size)
                    probs = F.softmax(logits, dim=-1)
                    topk_probs, topk_indices = torch.topk(probs, beam_width)
                    for i in range(beam_width):
                        token_id = topk_indices[0, i].item()
                        log_prob = topk_probs[0, i].item()
                        new_seq = torch.cat([seq, torch.tensor([[token_id]], device=device)], dim=1)
                        new_log_prob = log_prob + log_prob
                        new_beams.append((new_seq, new_log_prob))
                beams = sorted(new_beams, key=lambda x: x[1], reverse=True)[:beam_width]
                if all(seq[0, -1].item() == eos_token for seq, _ in beams):
                    break
            best_seq = max(beams, key=lambda x: x[1])[0].squeeze(0)
            outputs.append(best_seq)
    model.train()
    return outputs


def translate(model, seqs, max_len, device, en_stoi, zh_itos, sos_token, eos_token, method='beam_search', **kwargs):
    B = len(seqs)
    inputs = torch.zeros(B, max_len, dtype=torch.long, device=device)
    for i, seq in enumerate(seqs):
        tokens = tokenize_en(seq)
        encoded = encode(tokens, en_stoi)
        length = min(len(encoded), max_len)
        inputs[i, :length] = torch.tensor(encoded[:length], dtype=torch.long, device=device)
    if method == 'top_k_sample':
        outputs = top_k(model, inputs, sos_token, eos_token, max_len=max_len, device=device, **kwargs)
    elif method == 'beam_search':
        outputs = beam_search(model, inputs, sos_token, eos_token, max_len=max_len, device=device, **kwargs)
    results = []
    for i in range(B):
        result = decode(outputs[i], zh_itos)
        result = result.replace('<eos>', '').replace('|', '').replace('<sos>', '')
        results.append(result)
    return results

In [12]:
seqs = [
    "africa countries and europe countries, africa countries and europe countries.",
    "paris as the economic crisis deepens and widens, the world has been searching for historical analogies to help us understand what has been happening."
]
print("\n".join(seqs))


print("=" * 100)
args = [{
    "method": "top_k_sample",
    "top_k": 5,
    "temperature": 0.5
},
]

sos_token = zh_stoi['<sos>']
eos_token = zh_stoi['<eos>']

for arg in args:
    print(", ".join([f"{k}: {v}" for k, v in arg.items()]))
    print("\n".join(translate(model, seqs, max_len, device, en_stoi, zh_itos, sos_token, eos_token, **arg)))

africa countries and europe countries, africa countries and europe countries.
paris as the economic crisis deepens and widens, the world has been searching for historical analogies to help us understand what has been happening.
method: top_k_sample, top_k: 5, temperature: 0.5
非洲国家和欧洲国家非洲国家和欧洲国家.
巴黎随着经济危机的深化和不断加深,世界一直在寻找历史相似性来帮助我们理解已经发生的事情.
